# Funções Auxiliares e Imports

In [47]:
# !pip install community
# !pip install python-louvain

## Imports

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random as random
from scipy.stats import pearsonr
from scipy import stats
from scipy.linalg import expm
import pandas as pd
import seaborn as sns
from networkx.algorithms import community
from itertools import chain, combinations
from scipy.cluster.hierarchy import dendrogram
from networkx.algorithms.community import greedy_modularity_communities
from community import community_louvain


file_path = "C:/Users/gabri/Downloads/Redes Complexas/"

## Lendo e plotando o grafo dos dados

### Base Jazz

In [49]:
# Base de dados para a questão 1
jazz = nx.read_edgelist(file_path + "data/jazz.txt")

# plt.figure(figsize=(12,10))
# pos = nx.spring_layout(adv)
# nx.draw(adv, pos, node_color="lightgray", node_size=500, with_labels=True)

### Padronização da Rede

In [50]:
# Função que "padroniza" a rede, transformando-a em uma rede sem direção, e retornando o maior componente conectado para a análise.
def padronizar_rede(G):
    # Transformamos o grafo em uma rede sem direção.
    G = G.to_undirected()
    G.remove_edges_from(nx.selfloop_edges(G))

    # Vamos selecionar apenas o maior componente conectado.
    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
    G = G.subgraph(Gcc[0])

    # Transformando os labels para números inteiros, começando com 0:

    G = nx.convert_node_labels_to_integers(G, first_label=0)

    # Número de vértices e arestas:

    N = len(G)
    M = G.number_of_edges()
    print('Number of nodes:', N)
    print('Number of edges:', M)
    return G



## Funções do Questionário 1

### Cálculo do Grau Médio

In [51]:
# Função para calcular o grau médio do grafo.
def mean_degree(G):
    vk = dict(G.degree()).values()
    vk = np.array(list(vk))
    print('Degree', vk)



    md = np.mean(vk)
    # print('Mean degree: ', md)
    return md



### Cálculo da Distribuição do Grau

In [52]:
# Função para calcular a distribuição do grau do grafo.
def degree_distribution(G):
    vk = dict(G.degree())
    vk = list(vk.values())  # we get only the degree values
    vk = np.array(vk)
    maxk = np.max(vk)
    mink = np.min(vk)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk

# Função para calcular o momento da distribuição do grau do grafo.
def momment_of_degree_distribution(G,m):
    k,Pk = degree_distribution(G)
    M = sum((k**m)*Pk)
    return M

# Função para calcular a variância do grau.
def variance(G):
    variance = momment_of_degree_distribution(G,2) - momment_of_degree_distribution(G,1)**2
    return variance
# print("Variância do grau = ", variance)

### Cálculo do Momento do Grau

In [53]:
# Função para calcular o momento do grafo.
def momment(G,m):
    M = 0
    N = len(G)
    for i in G.nodes:
        M = M + G.degree(i)**m
    M = M/N
    return M

### Cálculo da Entropia de Shannon

In [54]:
# Função para calcular a entropia de Shannon do grafo.
def shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H

# Função para calcular a entropia de Shannon normalizada do grafo.
def normalized_shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H/np.math.log(len(G),2)

### Cálculo da Transitividade e do Coeficiente de Agrupamento

In [55]:
# Função para calcular o coeficiente de transitividade do grafo.
def graph_transitivity(G):
    CC = (nx.transitivity(G))
    # print("Transitivity = ","%3.4f"%CC)
    return "%3.4f"%CC

# Função para calcular o coeficiente de agrupamento do grafo.
def avg_clustering_coefficient(G):
    avc = nx.average_clustering(G)
    # print("Average clustering:", "%3.4f"%avc)
    return "%3.4f"%avc

## Funções do Questionário 2

### Distâncias

In [56]:
# Função para calcular e plotar a distribuição de grau da rede.
def graph_avg_distance(G):

    if nx.is_connected(G) == True:
        l = nx.average_shortest_path_length(G)
        print("Average shortest path length:", "%3.4f"%l)
    else:
        print("The graph has more than one connected component")
    return "%3.4f"%l
    

# Função para calcular o diâmetro do grafo.
def graph_diameter(G):
    d = nx.diameter(G)
    print('Network diameter:', d)
    return d

# Função para calcular a eficiência global e local do grafo.
def graph_efficiency(G):

    E = nx.global_efficiency(G)
    print('Network efficiency', E)

    leff = nx.local_efficiency(G)
    print('The average local efficiency of the network:', leff)

# Função para calcular a distância geodésica do grafo.
def graph_geodesic_distance(G):
    N = len(G)
    d = graph_diameter(G)

    if nx.is_connected(G) == True:
        D = np.zeros(shape=(N,N)) # D is the matrix of distances
        vl = []
        for i in np.arange(0,N):
            for j in np.arange(i+1, N):
                if(i != j):
                    aux = nx.shortest_path(G,i,j)
                    dij = len(aux)-1
                    D[i][j] = dij
                    D[j][i] = dij
                    vl.append(dij)
        x = range(0,d+1)
        plt.hist(vl, bins = x, density=True, color='#0504aa',alpha=0.7, rwidth=0.85)
        plt.title("Distribution of the geodesic distances", fontsize=20)
        plt.ylabel("P(l)", fontsize=15)
        plt.xlabel("Shortest path length (l)", fontsize=15)
        #plt.grid(True)
        plt.savefig('av_short_path.svg')
        plt.show(True)
    else:
        print("The graph has more than one connected component")



### Cálculo da Correlação de Grau a Grau

In [57]:
# Função para calcular e plotar a correlação de grau a grau (degree-degree correlation).
def degree_correlation(G):
    r=nx.degree_assortativity_coefficient(G)
    print("Assortativity = ","%3.4f"%r)

# Função para calcular a correlação de Pearson entre os graus dos nós.
def pearson_correlation(G):
    ki = []
    kj = []
    for i in range(0,len(G.nodes())):
        for j in range(0, len(G.nodes())):
            if(G.has_edge(i,j) == True):
                ki.append(G.degree(i))
                kj.append(G.degree(j))
                
    
    # calculate Pearson's correlation
    corr, _ = pearsonr(ki, kj)
    print('Pearsons correlation: %.3f' % corr)

# Função para calcular o grau médio de vizinhança (knn) do grafo.
def avg_neighbor_degree(G):

    knn = []
    for i in G.nodes():
        aux =  nx.average_neighbor_degree(G, nodes = [i])
        knn.append(float(aux[i]))
    knn = np.array(knn)
    print("Average degree of the neighborhood of the network:", "%3.2f"%np.mean(knn))
    return knn


# Função para calcular a correlação entre conectividade média da vizinhança
#  e grau do nó. Positiva em redes sortidas, negativa em redes assortativas.
def avc_assnet_degree(G):
    knn = avg_neighbor_degree(G)

    vk = dict(G.degree())
    vk = list(vk.values())



    knnk = list()
    ks = list()
    for k in np.arange(np.min(vk), np.max(vk)+1):
        aux = vk == k
        if(len(knn[aux]) > 0):
            av_knn = np.mean(knn[aux]) #average clustering among all the nodes with degree k
            knnk.append(av_knn)
            ks.append(k)
    fig= plt.figure(figsize=(10,6))

    plt.plot(ks, knnk, '-o', color='gray',markersize=10, linewidth=0,
            markerfacecolor='lightgray',
            markeredgecolor='black',
            markeredgewidth=2)
    #plt.loglog(ks,knnk,'bo',basex=10,basey=10)
    #plt.title("Average neighborhood connectivity vs degree")
    plt.ylabel("knn(k)", fontsize = 20)
    plt.xlabel("k", fontsize = 20)
    #plt.savefig('knnk.eps')

    # determine best fit line
    par = np.polyfit(ks, knnk, 1, full=True)
    slope=par[0][0]
    intercept=par[0][1]
    xl = [min(ks), max(ks)]
    yl = [slope*xx + intercept  for xx in xl]
    plt.plot(xl, yl, '--', linewidth=3, color='black')
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.savefig('knn.eps') #save the figure into a file
    plt.show(True)

    return ks, knnk

# Função para calcular o coeficiente de correlação de Pearson entre
#  o grau do nó e a conectividade média da vizinhança.
def pearson_correlation_knn(ks, knnk):
    rho = np.corrcoef(ks, knnk)[0,1]
    print('Pearson correlation coefficient:', rho)

# Função para calcular o coeficiente de correlação de Spearman entre
#  o grau do nó e a conectividade média da vizinhança.
def spearman_correlation_knn(ks, knnk):
    s = stats.spearmanr(ks, knnk)
    print('Spearman rank correlation coefficient:', s[0])
    print('p-value:', s[1])

### Caminhadas Aleatórias

In [58]:
# Função para realizar caminhadas aleatórias no grafo
def random_walk(G, T, seed_node):
    
    # neighbors
    ng = G.neighbors(seed_node)
    walk = []
    for t in range(0,T):
        next_node = random.choice(list(ng))
        ng = G.neighbors(next_node)
        walk.append(next_node)
    print('Walk:', walk)



    nx.draw(G, with_labels = True, node_size=500, font_size=16, pos = nx.spring_layout(G))
    plt.show(True)

    A = nx.to_numpy_array(G)
    print(A)
    print('Number of walks of length two:')
    print(A@A)



## Funções do Questionário 3

### Funções de Centralidades

In [59]:
# Função para calcular a centralidade de grau do grafo.
# Degree centrality: C(i) = k(i)/(N-1)
def degree_centrality(G):
    vk = dict(G.degree())
    vk = list(vk.values())
    print('Degree centrality', vk)
    return vk

# Função para plotar a distribuição da centralidade de grau do grafo.
def degree_centrality_distribution(vk):
    plt.figure(figsize=(6,4))
    plt.hist(vk, density=True)
    plt.title("Distribution of the Degree Centrality", fontsize=20)
    plt.ylabel("P(k)", fontsize=20)
    plt.xlabel("Degree Centrality (k)", fontsize=20)
    #plt.grid(True)
    plt.savefig('degree-centrality.eps')
    plt.show(True)

# Função para calcular a centralidade de proximidade do grafo.
# Closeness centrality: C(i) = 1/(sum_{j=1}^{N} d(i,j))
def closeness_centrality(G):
    clc = dict(nx.closeness_centrality(G))
    print('Closeness centrality', clc)

    clc = list(clc.values())
    av_clc = np.mean(clc)
    print('Average closeness centrality', av_clc)

    return clc, av_clc

# Função para plotar a distribuição da centralidade de proximidade do grafo.
def closeness_centrality_distribution(clc):
    # Distribuição de Closeness Centrality
    plt.figure(figsize=(6,4))
    plt.hist(clc, density=True)
    plt.title("Distribution of the Closeness Centrality", fontsize=20)
    plt.ylabel("P(CLC)", fontsize=20)
    plt.xlabel("Closeness Centrality (CLC)", fontsize=20)
    plt.savefig('closeness.eps')
    plt.show(True)

    
# Função para calcular a centralidade de intermediação do grafo.
# Betweenness centrality: C(i) = sum_{s!=i!=t} (sigma(s,t|i)/sigma(s,t))
def betweenness_centrality(G):
    btc = dict(nx.betweenness_centrality(G))
    print('Betweenness centrality', btc)

    btc = list(btc.values())
    av_btc = np.mean(btc)
    print('Average betweenness centrality', av_btc)

    return btc, av_btc

# Função para plotar a distribuição da centralidade de intermediação do grafo.
def betweenness_centrality_distribution(btc):
    
    # Distribuição de Betweenness Centrality
    plt.figure(figsize=(6,4))
    plt.hist(btc, density=True)
    plt.title("Distribution of the Betweenness Centrality", fontsize=20)
    plt.ylabel("P(BTC)", fontsize=20)
    plt.xlabel("Betweenness Centrality (BTC)", fontsize=20)
    plt.savefig('betweenness.eps')
    plt.show(True)

    
# Função para calcular a centralidade de autovetor do grafo.
# Eigenvector centrality: C(i) = sum_{j=1}^{N} (A(i,j)*C(j))
def eigenvector_centrality(G):
    evec = dict(nx.eigenvector_centrality(G))
    print('Eigenvector centrality', evec)

    evec = list(evec.values())
    av_evec = np.mean(evec)
    print('Average eigenvector centrality', av_evec)

    return evec, av_evec
    
# Função para plotar a distribuição da centralidade de autovetor do grafo.
# A distribuição é feita em relação ao valor absoluto da centralidade de autovetor.
def eigenvector_centrality_distribution(evec):
    # Distribuição de Eigenvector Centrality
    plt.figure(figsize=(6,4))
    plt.hist(evec, density=True)
    plt.title("Distribution of the Eigenvector Centrality", fontsize=20)
    plt.ylabel("P(EVC)", fontsize=20)
    plt.xlabel("Eigenvector Centrality (EVC)", fontsize=20)
    plt.savefig('eigenvector.eps')
    plt.show(True)

    
# Função para calcular a centralidade de PageRank do grafo.
# PageRank centrality: C(i) = (1-d)/N + d*sum_{j=1}^{N} (A(i,j)*C(j)/k(j))
def pagerank(G):
    pr = dict(nx.pagerank(G, alpha=0.85))
    print('PageRank: ', pr)

    pr = list(pr.values())
    pr = np.array(pr)
    av_pr = np.mean(pr)
    print('Average PageRank centrality', av_pr)

    B = dict(nx.betweenness_centrality(G))
    B = list(B.values())
    B = np.array(B)
    plt.figure(figsize=(6,4))
    plt.ylabel("Betweennees centrality", fontsize = 20)
    plt.xlabel("Page Rank", fontsize = 20)
    plt.plot(pr, B, 'ro')
    plt.show(True)

    return pr, av_pr

# Função para plotar a distribuição da centralidade de PageRank do grafo.
# A distribuição é feita em relação ao valor absoluto da centralidade de PageRank.
def pagerank_distribution(pr):
    # Distribuição de PageRank Centrality
    plt.figure(figsize=(6,4))
    plt.hist(pr, density=True)
    plt.title("Distribution of the PageRank Centrality", fontsize=20)
    plt.ylabel("P(PRC)", fontsize=20)
    plt.xlabel("PageRank Centrality (PRC)", fontsize=20)
    plt.savefig('pagerank.eps')
    plt.show(True)

    
# Função para calcular a centralidade de Katz do grafo.
# Katz centrality: 
# C(i) = sum_{j=1}^{N} (A(i,j)*C(j)) + alpha*sum_{j=1}^{N} (A(i,j)*C(j))
def k_core(G):
    KC = dict(nx.core_number(G))
    print('K-core: ', KC)
    KC = list(KC.values())
    KC = np.array(KC)

    return KC

# Função para plotar a distribuição da centralidade de K-core do grafo.
# A distribuição é feita em relação ao valor absoluto da centralidade de K-core.
def k_core_distribution(KC):
    # Distribuição de K-core
    plt.figure(figsize=(6,4))
    plt.hist(KC, density=True)
    plt.title("Distribution of the K-core", fontsize=20)
    plt.ylabel("P(KC)", fontsize=20)
    plt.xlabel("K-core (KC)", fontsize=20)
    plt.savefig('k-core.eps')
    plt.show(True)

    
# Função para plotar a rede.
def plotting_graph(G, node_size, node_color, with_labels):
    plt.figure(figsize=(12,10))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, node_color=node_color, 
            node_size=node_size, with_labels=with_labels)
    plt.show(True)


### Acessibilidade de Caminhadas Aleatórias

In [60]:
# Função para calcular a acessibilidade do grafo.
def acc(G):
    N = len(G.nodes())
    vk = dict(G.degree())
    vk = list(vk.values())
    A = nx.adjacency_matrix(G)
    P = np.zeros((N,N), dtype = 'float')
    for i in np.arange(0, N):
        for j in np.arange(0, N):
            if(vk[i] > 0):
                P[i,j] = A[i,j]/vk[i]
    P2 = expm(P)/np.exp(1)
    vacc = np.zeros(N, dtype = float)
    for i in np.arange(0, N):
        acc = 0
        for j in np.arange(0,N):
            if(P2[i,j] > 0):
                acc = acc + P2[i,j]*np.log(P2[i,j])
        acc = np.exp(-acc)
        vacc[i] = acc
    
    print('ACC: ', vacc)


    plt.figure(figsize=(6,6))
    pos=nx.spring_layout(G)
    nx.draw(G, with_labels = True, pos = pos)
    plt.show(True)  

    return vacc


# Função para calcular a acessibilidade do grafo de acordo com o 
# modelo de Hub and Leaves.
def hub_acc(N):
    x = np.cosh(1)/np.exp(1)
    y = np.sinh(1)/np.exp(1)
    acchub = np.exp(-x*np.log(x) -y*np.log(y/(N-1)))
    print('Hub acc:', acchub)
    # Accessibility of the leaves
    x = np.sinh(1)/np.exp(1)
    y = (np.cosh(1)-1)/(np.exp(1)*(N-1))
    accleaves = np.exp(-x*np.log(x) - (N-2)*y*np.log(y) \
                       - (1/np.exp(1) + y)*np.log(1/np.exp(1) + y))
    print('Leaves acc:', accleaves)

    return acchub, accleaves

### Correlação entre medidas de centralidade

In [61]:
# Função para calcular a matriz de correlações entre as centralidades.
def centralities_df(G):
    vk = degree_centrality(G)
    clc, av_clc = closeness_centrality(G)
    btc, av_btc = betweenness_centrality(G)
    evec, av_evec = eigenvector_centrality(G)
    pr, av_pr = pagerank(G)
    KC = k_core(G)
    df = pd.dataframe({'Degree': vk, 'Closeness': clc, 'Betweenness': btc, \
                    'Eigenvector': evec, 'PageRank': pr, 'K-core': KC})
    
    return df

# Função para plotar a matriz de correlação entre as centralidades.
def correlation_matrix(df):
    corr = df.corr()
    #Plot Correlation Matrix using Matplotlib
    plt.figure(figsize=(8,8))
    plt.imshow(corr, cmap='Blues', interpolation='none', aspect='auto')
    plt.colorbar()
    plt.xticks(range(len(corr)), corr.columns, rotation='vertical', fontsize=20)
    plt.yticks(range(len(corr)), corr.columns, fontsize=20)
    plt.suptitle('Correlation between centrality measures', fontsize=20)
    plt.grid(False)
    plt.show()

# Função para plotar a matriz de correlação entre as centralidades usando o Seaborn.
def correlogram(df):
    sns.pairplot(df)
    plt.show(True)

# Função para plotar o mapa de calor da matriz de correlação entre as centralidades.
def heatmap(df):
    corr = df.corr()
    plt.figure(figsize=(7, 7))
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):ax = sns.heatmap(corr, mask=mask, vmax=1, square=True)
    plt.show()

## Funções do Questionário 4

In [ ]:
# Função para plotar o dendrograma das comunidades da rede.
def girvan_newman_dendogram(G):
    communities = list(community.centrality.girvan_newman(G))
    
    node_id = 0
    init_node2community_dict = {node_id: communities[0][0].union(communities[0][1])}
    for comm in communities:
        for subset in list(comm):
            if subset not in init_node2community_dict.values():
                node_id += 1
                init_node2community_dict[node_id] = subset

    # turning this dictionary to the desired format 
    node_id_to_children = {e: [] for e in init_node2community_dict.keys()}
    for node_id1, node_id2 in combinations(init_node2community_dict.keys(), 2):
        for node_id_parent, group in init_node2community_dict.items():
            if len(init_node2community_dict[node_id1].intersection(init_node2community_dict[node_id2])) == 0 and group == init_node2community_dict[node_id1].union(init_node2community_dict[node_id2]):
                node_id_to_children[node_id_parent].append(node_id1)
                node_id_to_children[node_id_parent].append(node_id2)

    # recording node_labels dict for the correct label for dendrogram leaves
    node_labels = dict()
    for node_id, group in init_node2community_dict.items():
        if len(group) == 1:
            node_labels[node_id] = list(group)[0]
        else:
            node_labels[node_id] = ''

    # needing a subset to rank dict to later know within all k-length merges which came first
    subset_rank_dict = dict()
    rank = 0
    for e in communities[::-1]:
        for p in list(e):
            if tuple(p) not in subset_rank_dict:
                subset_rank_dict[tuple(sorted(p))] = rank
                rank += 1
    subset_rank_dict[tuple(sorted(chain.from_iterable(communities[-1])))] = rank

    # a function to get a merge height so that it is unique
    def get_merge_height(sub):
        sub_tuple = tuple(sorted([node_labels[i] for i in sub]))
        n = len(sub_tuple)
        other_same_len_merges = {k: v for k, v in subset_rank_dict.items() if len(k) == n}
        min_rank, max_rank = min(other_same_len_merges.values()), max(other_same_len_merges.values())
        range = (max_rank-min_rank) if max_rank > min_rank else 1
        return float(len(sub)) + 0.8 * (subset_rank_dict[sub_tuple] - min_rank) / range

    G           = nx.DiGraph(node_id_to_children)
    nodes       = G.nodes()
    leaves      = set( n for n in nodes if G.out_degree(n) == 0 )
    inner_nodes = [ n for n in nodes if G.out_degree(n) > 0 ]

    # Compute the size of each subtree
    subtree = dict( (n, [n]) for n in leaves )
    for u in inner_nodes:
        children = set()
        node_list = list(node_id_to_children[u])
        while len(node_list) > 0:
            v = node_list.pop(0)
            children.add( v )
            node_list += node_id_to_children[v]
        subtree[u] = sorted(children & leaves)

    inner_nodes.sort(key=lambda n: len(subtree[n])) # <-- order inner nodes ascending by subtree size, root is last

    # Construct the linkage matrix
    leaves = sorted(leaves)
    index  = dict( (tuple([n]), i) for i, n in enumerate(leaves) )
    Z = []
    k = len(leaves)
    for i, n in enumerate(inner_nodes):
        children = node_id_to_children[n]
        x = children[0]
        for y in children[1:]:
            z = tuple(sorted(subtree[x] + subtree[y]))
            i, j = index[tuple(sorted(subtree[x]))], index[tuple(sorted(subtree[y]))]
            Z.append([i, j, get_merge_height(subtree[n]), len(z)]) # <-- float is required by the dendrogram function
            index[z] = k
            subtree[z] = list(z)
            x = z
            k += 1

    # dendrogram
    plt.figure()
    fig= plt.figure(figsize=(15,8))
    dendrogram(Z, labels=[node_labels[node_id] for node_id in leaves], leaf_rotation=0)
    plt.savefig('dendrogram.eps')

### Métodos para Identificação de Comunidades

In [ ]:
# Função para detectar comunidades na rede usando o algoritmo de Girvan-Newman.
def girvan_newman(G):
    communities = community.centrality.girvan_newman(G)
    k = 2
    for i in range(0, k-1):
        next_level_communities = next(communities)
    c = sorted(map(sorted, next_level_communities))
    for cl in c:
        print('community:', cl)
    return c

# Função para plotar as comunidades detectadas pelo algoritmo de Girvan-Newman.
def girvan_newman_plot(G, c):
    colors = ['black', 'gray', 'green', 'c', 'm', 'y', 'w']

    pos = nx.spring_layout(G)
    fig= plt.figure(figsize=(10,6))
    nx.draw(G, pos=pos, node_color = 'white', edge_color='lightgray')
    aux = 0
    for cm in c:
        nx.draw(G.subgraph(cm), pos=pos, node_color = colors[aux], 
                        with_labels = True, node_size=300, font_color = 'white')
        aux = aux + 1
    plt.savefig('girvan_newman.eps') #save the figure into a file
    plt.show(True)

# Função para detectar comunidades na rede usando o algoritmo de Fast Greedy.
def fast_greedy(G):
    c = list(greedy_modularity_communities(G))
    communities = np.zeros(len(G.nodes()))
    nc = 0
    for k in range(0,len(c)):
        communities[sorted(c[k])]=nc
        nc = nc+1
        print('Community:', sorted(c[k]))
    return c, communities
    
# Função para plotar as comunidades detectadas pelo algoritmo de Fast Greedy.
def fast_greedy_plot(G, communities):

    colors = ['red', 'blue', 'green', 'black', 'magenta', 'yellow', 'white']

    pos = nx.spring_layout(G)
    fig= plt.figure(figsize=(10,6))
    nx.draw(G, pos=pos, node_color = 'white', edge_color='lightgray', style='dashed')
    aux = 0
    for cm in communities:
        nx.draw(G.subgraph(cm), pos=pos, node_color = colors[aux], 
                        with_labels = True, node_size=300, font_color = 'white')
        aux = aux + 1
    plt.savefig('fast_greedy.eps') #save the figure into a file
    plt.show(True)

# Função para detectar comunidades na rede usando o algoritmo de Louvain.
def louvain(G):
    partitions = community_louvain.best_partition(G)
    return partitions

# Função para plotar as comunidades detectadas pelo algoritmo de Louvain.
def louvain_plot(G, partitions):
    pos = nx.spring_layout(G)
    fig= plt.figure(figsize=(10,6))
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    size = float(len(set(partitions.values())))
    count = 0
    for com in set(partitions.values()) :
        count = count + 1.
        list_nodes = [nodes for nodes in partitions.keys() if partitions[nodes] == com]
        nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 50, node_color = colors[int(count)])
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    plt.show()

### Medida de Modularidade

In [ ]:
# Função para calcular a modularidade de uma rede
def modularity(G, communities):
    A = nx.adjacency_matrix(G)
    N = len(G)
    M = G.number_of_edges()
    Q = 0
    for i in np.arange(0,N):
        ki = len(list(G.neighbors(i)))
        for j in np.arange(0,N):
            if(communities[i]==communities[j]):
                kj = len(list(G.neighbors(j)))
                Q = Q + A[i,j]-(ki*kj)/(2*M)
    Q = Q/(2*M)
    return Q


### Benchmarks

In [ ]:
# Função para criar uma rede de benchmark com o algoritmo Gilvan-Newman.
def benchmark_gilvan_newman(k, kout):
    kin = k - kout
    pin = kin/32
    pout = kout/(128-32)
    G = nx.random_partition_graph([32,32,32,32],pin,pout)
    return G

# Função para plotar a rede de benchmark com o algoritmo Gilvan-Newman.
def plot_benchmark_gilvan_newman(G):
    fig= plt.figure(figsize=(10,6))
    pos=nx.spring_layout(G)
    nx.draw(G, with_labels = False, edge_color='gray', 
            node_size=100, font_size=16,  width=1, pos = pos)
    plt.show(True)

# Função para criar uma rede de benchmark com o algoritmo de Lancichinetti.
def benchmark_lancichinetti(N, tau1, tau2, mu, k, minc, maxc, seed):
    G = nx.LFR_benchmark_graph(n = N, tau1 = tau1, tau2 = tau2, 
                               mu = mu, min_degree = k, 
                               max_degree = k, min_community=minc, 
                               max_community = maxc, seed = seed)
    return G

# Função para plotar a rede de benchmark com o algoritmo de Lancichinetti.
def plot_benchmark_lancichinetti(G):
    fig= plt.figure(figsize=(10,6))
    pos=nx.spring_layout(G)
    nx.draw(G, with_labels = False, edge_color='gray', 
            node_size=100, font_size=16,  width=1, pos = pos)
    plt.show(True)

# Algoritmo de detecção de comunidades para redes de benchmark.
def benchmark_communities(G):
    c = {frozenset(G.nodes[v]['community']) for v in G}
    communities = np.zeros(len(G.nodes()))
    cl = 0
    for a in c:
        b = list(a)
        communities[b] = cl
        #print(cl,':',sort(b))
        cl = cl + 1
    return communities

# Questões

Use:


G = G.to_undirected()

G.remove_edges_from(nx.selfloop_edges(G))

Gcc = sorted(nx.connected_components(G), key=len, reverse=True)

G = G.subgraph(Gcc[0])

G = nx.convert_node_labels_to_integers(G, first_label=0)

## Questão 1)

Calcule a modularidade para a rede Jazz usando método fastgreedy.


In [ ]:
# Padronizo a rede para a análise
jazz = padronizar_rede(jazz)

# Calculo a modularidade da rede utilizando o método de Fast Greedy
c, communities = fast_greedy(jazz)

print(f"Modularity: {modularity(jazz, communities)}")

# plot_dendogram(list(communities))
print(list(communities))
print(communities)
print(list(c))
print(c)
print(list(greedy_modularity_communities(jazz)))
print(list(community.centrality.girvan_newman(jazz)))

Number of nodes: 198
Number of edges: 2742
Community: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 100, 111, 120, 125, 126, 127, 128, 129, 130, 132, 134, 135, 136, 144, 145, 146, 147, 148, 149, 150, 151, 152, 157, 158, 159, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 191, 194]
Community: [24, 25, 26, 27, 47, 53, 54, 73, 74, 75, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 131, 133, 137, 139, 143, 154, 156, 161, 162, 190, 192, 193, 197]
Community: [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 76, 77, 78, 138, 140, 141, 142, 153, 155, 160, 179, 180, 181, 182, 183, 184, 185, 189, 195, 196]
Community: [186, 187, 188]
Modularity: 0.438907815375376

## Questão 2)

Calcule a correlação de Pearson entre a medida betweeness centrality e grau para a rede hamsterster

In [67]:
# Calculo a modularidade da rede utilizando o método de Louvain
partitions = louvain(jazz)

print(f"Modularity: {modularity(jazz, partitions)}")


Modularity: 0.43723901266251625


## Questão 3)

Considere o método de geração de redes LFR_benchmark_graph. Obtenha os valores da modularidade para μ=0.05,μ=0.1,μ=0.2

. Use o código a seguir para gerar as redes. Use o algoritmo de Louvain.

N = 128

tau1 = 3

tau2 = 1.5

k =16

minc = 32

maxc = 32

G = nx.LFR_benchmark_graph(n = N, tau1 = tau1, tau2 = tau2, mu = mu, min_degree = k, max_degree = k, min_community=minc, max_community = maxc, seed = 10)

In [68]:
# Defino os parâmetros para uma rede benchmark
N = 128
tau1 = 3
tau2 = 1.5
k = 16
minc = 32
maxc = 32
mus = [0.05, 0.1, 0.2]

# Para cada valor de mu, gero uma rede benchmark com o algoritmo Lanchinetti
for mu in mus:

    benchmark = benchmark_lancichinetti(N, tau1, tau2, mu, k, minc, maxc)
    
    # Utilizo o método de Louvain de detecção de comunidades para
    # encontrar as comunidades da rede benchmark

    communities = louvain(benchmark)

    # Calculo a modularidade da rede benchmark
    print(f"mu: {mu}    ;    Modularity: {modularity(benchmark, communities)}")

# Para cada valor de mu, gero uma rede benchmark com o algoritmo Lanchinetti
# for mu in mus:

#     benchmark = benchmark_lancichinetti(N, tau1, tau2, mu, k, minc, maxc)

#     # Utilizo um algoritmo de detecção de comunidades para 
#     # encontrar as comunidades da rede benchmark
#     communities = benchmark_communities(benchmark)

#     # Calculo a modularidade da rede benchmark
#     print(f"mu: {mu}    ;    Modularity: {modularity(benchmark, communities)}")


    

mu: 0.05    ;    Modularity: 0.643639535397838
mu: 0.1    ;    Modularity: 0.5427818606053953
mu: 0.2    ;    Modularity: 0.44487909975820294


## Questão 4) 

Considere o método de geração de redes LFR_benchmark_graph. Obtenha os valores da modularidade para μ=0.05,μ=0.2,μ=0.4. Use o código a seguir para gerar as redes. Use o algoritmo fastgreedy.

N = 128

tau1 = 3

tau2 = 1.5

k =16

minc = 32

maxc = 32

G = nx.LFR_benchmark_graph(n = N, tau1 = tau1, tau2 = tau2, mu = mu, min_degree = k, max_degree = k, min_community=minc, max_community = maxc, seed = 10)

In [69]:
# Gero uma rede benchmark utilizando o método de Lanchinetti
N = 128
tau1 = 3
tau2 = 1.5
k = 16
minc = 32
maxc = 32
mus = [0.05, 0.2, 0.4]

# Para cada valor de mu, gero uma rede benchmark com o algoritmo Lanchinetti
for mu in mus:
    
    benchmark = benchmark_lancichinetti(N, tau1, tau2, mu, k, minc, maxc)

    # Utilizo o método de Fast Greedy de detecção de comunidades para
    # encontrar as comunidades da rede benchmark
    c, communities = fast_greedy(benchmark)

    # Calculo a modularidade da rede benchmark
    print(f"mu: {mu}    ;    Modularity: {modularity(benchmark, communities)}")

    
# Para cada valor de mu, gero uma rede benchmark com o algoritmo Lanchinetti
# for mu in mus:

#     benchmark = benchmark_lancichinetti(N, tau1, tau2, mu, k, minc, maxc)

#     # Utilizo um algoritmo de detecção de comunidades para 
#     # encontrar as comunidades da rede benchmark

#     communities = benchmark_communities(benchmark)

#     # Calculo a modularidade da rede benchmark
#     print(f"mu: {mu}    ;    Modularity: {modularity(benchmark, communities)}")



Community: [1, 6, 7, 10, 21, 22, 25, 31, 32, 37, 39, 44, 45, 46, 48, 52, 57, 60, 62, 64, 66, 73, 76, 85, 94, 101, 107, 110, 113, 115, 118, 123]
Community: [0, 3, 26, 27, 28, 35, 38, 40, 43, 49, 50, 51, 55, 56, 70, 79, 80, 83, 84, 86, 87, 92, 95, 102, 103, 106, 117, 119, 120, 121, 122, 124]
Community: [5, 8, 9, 11, 23, 30, 34, 47, 54, 58, 65, 68, 71, 72, 74, 75, 77, 81, 88, 89, 90, 91, 93, 97, 98, 104, 105, 109, 114, 116, 125, 126]
Community: [2, 4, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 29, 33, 36, 41, 42, 53, 59, 61, 63, 67, 69, 78, 82, 96, 99, 100, 108, 111, 112, 127]
mu: 0.05    ;    Modularity: 0.643639535397838
Community: [0, 3, 26, 27, 28, 35, 38, 40, 43, 49, 50, 51, 55, 56, 70, 79, 80, 83, 84, 86, 87, 92, 95, 102, 103, 106, 117, 119, 120, 121, 122, 123, 124]
Community: [5, 8, 9, 11, 23, 30, 34, 47, 54, 58, 65, 68, 71, 72, 74, 75, 77, 81, 85, 88, 89, 90, 91, 93, 97, 98, 104, 105, 109, 114, 116, 125, 126]
Community: [2, 4, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 29, 33, 36, 41, 4